# Imports

In [1]:
# IMPORTS
import copy
import numpy as np
from enum import Enum
import bisect 
import time

# CLASS INITIALIZATIONS

In [2]:
class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2

class Coordinate():
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  # Primarily for testing to see if Coordinate works
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

#
# CAR
# 

class Car(object):
  def __init__(self, letter, coordinate, fuel):
    self.letter = letter  # Remember that A is the most important
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = int(fuel)

  def isA(self):
    return self.letter == 'A'

  def move(self, amt):
    if self.direction == Direction.HORIZONTAL:
      for c in self.coordinates:
        c.Y += amt
    else:
      for c in self.coordinates:
        c.X += amt
    self.fuel -= abs(amt)
  
  def canMove(self):
    return self.fuel > 0

  def compareLetters(self, letter):
    return self.letter == letter

  def addCoordinate(self, newCoord):
    self.coordinates.append(newCoord)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X == newCoord.X and self.coordinates[0].Y != newCoord.Y:
      self.direction = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.direction = Direction.VERTICAL

  def getLetter(self):
    return self.letter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.letter + " -> ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.direction)

  @staticmethod
  def create(letter, coordinates, fuel):
    return Car(letter, coordinates, fuel)


#
# BOARD
# 

class Board(object):
  def __init__(self, cars):
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    self.cars = []
    for car in cars:  
      c = car
      self.cars.append(copy.deepcopy(c));
      for coord in c.coordinates:
        self.grid[coord.X][coord.Y] = car.letter
        
  def __eq__(self, obj):
        return self.grid == obj.grid

  def findCarPaths(self, car):
    if car.canMove():
      fwd = self.checkPathForward(car)
      bwd = self.checkPathBackward(car)
      carPath = bwd + fwd
    else:
      carPath = []
    return carPath
  
  def moveCar(self, carLetter, amt):
    for car in self.cars: 
      if car.letter == carLetter:
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = "."
        car.move(amt)
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = carLetter
        
        # Is this a regular car that just left?
        if car.letter == self.grid[2][5] and car.direction == Direction.HORIZONTAL and car.letter != "A":
          for coord in car.coordinates:
            self.grid[coord.X][coord.Y] = "."
          self.cars.remove(car)  
        
        
        direction="";
        if(amt<0 and car.direction==Direction.VERTICAL):
            direction="up"
        if(amt>0 and car.direction==Direction.VERTICAL):
            direction="down"
        if(amt<0 and car.direction==Direction.HORIZONTAL):
            direction="left"
        if(amt>0 and car.direction==Direction.HORIZONTAL):
            direction="right"        
        #returns the move performed
        return((carLetter, direction, abs(amt), car.fuel, self.stringLine()))
        
  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 4
  # range(5,6) -> 5
  # next gridPos = (1,5)
  # append -> 5 - 4 = 1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 1
  # range(2,6) -> 2, 3, 4, 5
  # next gridPos = (2,3)
  # append -> 2 - 1 = 1
  def checkPathForward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[len(car.coordinates)-1]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint+1, 6):
        #fuel req
      if(car.fuel < i-startPoint):
        continue;
      # Find next position to check    
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i] 
      else:
        gridPos = self.grid[i][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint)
      else:
        break;

    return possibleMoves

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 3
  # range(3,0,-1) -> 3, 2, 1
  # i = i - 1 -> 2, 1, 0
  # next gridPos = (1,2)
  # append -> 2 - 3 = -1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 0
  # range(0,0) -> NONE
  # next gridPos = NONE
  def checkPathBackward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[0]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint, 0, -1):
      if (car.fuel <= (startPoint-i)):
        continue
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i-1] 
      else:
        gridPos = self.grid[i-1][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint-1)
      else:
        break;

    return possibleMoves

  def winConditionMet(self):
    if self.grid[2][5] == "A":
      return True      
    return False

  def __str__(self): 
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
      s += "\n" 
    return s

  def stringLine(self):
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
    return s

  def AllUsedCars(self):
    s = ""
    for c in cars:
      if(c.fuel < 100):
        s += c.getLetter + " " + c.fuel + " ";
    return s


  #Heuristic definitions:

  #h1 = The number of blocking vehicles
  def h1(self):
        count = 0
        last = " "
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
        return count
  
  #h2 = The number of blocked positions
  def h2(self):
        count = 0
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e != ".":
                count += 1
        return count
    
  #h3 = The number of blocking vehicles (h1) multiplied by a constant
  def h3(self, constant):
        return self.h1()*constant
    
  #h4 = 
  def h4(self):
        count = 0
        last = " "
        blocked = []
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
                #Check if car is horizontal by looking at the next slot in the grid
                if self.grid[2][5-i-1] == e:
                    continue
                #If the car is not horizontal, we count how many squares block it in both directions
                else:
                    last2 = " "
                    up = 0
                    for j in range(2):
                        if self.grid[2-j-1][5-i] == e or self.grid[2-j-1][5-i] == "." or self.grid[2-j-1][5-i] == last2:
                            continue
                        else:
                            up += 1
                            last2 = self.grid[2-j-1][5-i]
                    if up == 0:
                        continue
                    last2 = " "
                    down = 0
                    for j in range(3):
                        if self.grid[2+j+1][5-i] == e or self.grid[2+j+1][5-i] == "." or self.grid[2+j+1][5-i] == last2:
                            continue
                        else:
                            down += 1
                            last2 = self.grid[2+j+1][5-i]
                    if down == 0:
                        continue
                    blocked.append(min(up, down))
                        
        if blocked:
            return count + min(blocked)
        else:
            return count
                    
                
        
        if self.grid[2][5] != "." and self.grid[2][5] != "A":
            return 1
        else:
            return 0

# Extracting Data From Sample-input.txt

In [3]:
demos = []
demoFuels = []

with open('Sample\\sample-input-50.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)

[['Z.BOOOZ.B....AAC...D.C...DPPPE.....E'], ['....ZZ....O..AA.OB....OB.....C...DDC'], ['....Z.....Z.AA.BC..O.BC..ODDEE.O....'], ['..ZOOO..ZP.B.AAP.B...P.......C.....C'], ['ZZ..O...BBOC..AAOC...DD...EFF...E...'], ['...O.....O.Z.AAO.Z......B.C...B.CPPP'], ['.....Z..OOOZAA.B.C...B.C.DDPPP......'], ['ZOOO..Z.BC.DAABC.D..PPP.............'], ['ZZBBCC....DE..AADE..O.....OFGG..OF..'], ['ZZ.BCCOOOB.D..EAADFFEGGHIJJ..HI...KK'], ['...ZZ....BCCAA.B.D...E.D...EFF......'], ['OZBBCCOZ.DEEOAADF.GGH.FP..HIIP.JJKKP'], ['OZBCCDOZBEFDOAAEF....GHHII.G..JJPPP.'], ['Z.BOOOZ.BCDDPAACEFPG..EFPGQQQH.....H'], ['...OOO....P...AAP..QQQP...ZBCC..ZB..'], ['Z.O...Z.O...AAO..........BBC.DPPPC.D'], ['..OPPP..OZ.QAAOZBQ...RBQ.CCRD....RD.'], ['ZZ..O.BCDDOEBCAAOE.P.....PF.GG.PFQQQ'], ['..Z.BB..ZC...AACD.....D...E.FF..E...'], ['...OOO......AA.P..ZZ.PB..C.PB..C.QQQ'], ['..ZBCC..ZB....AADE....DE.FFOOO......'], ['.ZZ.BB...CC.AA.OD....OD....OEE......'], ['OOO.P...ZZP...AAP.......BBC.DD..C...'], ['ZBBCDDZ..CEEAA..O..PPPOF..G.OFH

# Setting the boards up

In [4]:
boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if len(data) > 1:
    fuels.append(data[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(row,col))
          unique = False

      if unique:
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmt = 100
        if len(fuels) > 0:
          for f in fuels[0]:
            if current == f[0]:
              fuelAmt = f[1:]

        cars.append(Car.create(current, Coordinate(row,col), fuelAmt));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()
  
  board = Board(cars)
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)


  for b in boards:
    for c in b.cars:
      print(c)
    print()

CARS: 
--------
Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
C -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
D -> (3, 1)(4, 1) | 100 | Direction.VERTICAL
P -> (4, 2)(4, 3)(4, 4) | 100 | Direction.HORIZONTAL
E -> (4, 5)(5, 5) | 100 | Direction.VERTICAL

BOARD: 
--------
Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E


Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
C -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
D -> (3, 1)(4, 1) | 100 | Direction.VERTICAL
P -> (4, 2)(4, 3)(4, 4) | 100 | Direction.HORIZONTAL
E -> (4, 5)(5, 5) | 100 | Direction.VERTICAL

CARS: 
--------
Z -> (0, 4)(0, 5) | 100 | Direction.HORIZONTAL
O -> (1, 4)(2, 4)(3, 4) | 100 | Direction.VERTICAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
B ->

# Building State Space Search - Uniform Cost Search (UCS)

In [5]:
class Node(object):
    def __init__(self, board, cost):
        self.board = board
        self.cost = cost
        self.path = []
        self.parent = 0
        self.search = ""
    def __lt__(self, other):
        return self.cost < other.cost
    def __eq__(self, other):
        return self.board == other.board
  
    
class UCS(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0

    def recursivePrinting(self, node):
        if node.cost != 0:
            self.recursivePrinting(node.parent)  
        print(node.board)

    def search(self, board):
        self.opened = []
        self.closed = []
        
        self.count += 1
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0)
        self.opened.append(a)
        a.search = "0 0 0 " + a.board.stringLine()

        start_time = time.time()
        statessearched = 0
        
        f = open("Results\\ucs-search-"+ str(self.count) +".txt", "w")

        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\ucs-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                
                with open("Results\\all_results.txt", "a") as f:
                    f.write(str(self.count) + "\t\t\tUCS\t\tNA\t\tNA\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close()
                
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)
            
            #print(curr.search)
            f.write(curr.search + "\n")

            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()

                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                #self.recursivePrinting(curr)
                print("Solution path: ")
                for move in curr.path:
                    print(move[0]+ " "+move[1] + " " + str(move[2])+"; ", end='')

                ######

                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\ucs-sol-"+ str(self.count) +".txt", "w")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....
                f.write("\nSolution path: ")
                for move in curr.path:
                    f.write(move[0]+ " "+move[1] + " " + str(move[2])+"; ")

                f.write("\n\n");
                for move in curr.path:
                    f.write(move[0]+ "\t"+move[1] + "\t" + str(move[2]) + "\t" + str(move[3]) + "\t" + str(move[4])+"\n")

                f.write("\n" + str(curr.board) + "\n")
                f.close()
                
                with open("Results\\all_results.txt", "a") as f:
                    f.write(str(self.count) + "\t\t\tUCS\t\tNA\t\t" + str(curr.cost) + "\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close()

                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode = Node(Board(curr.board.cars), curr.cost+1)   
                        newnode.parent = curr
                        newnode.path = copy.deepcopy(newnode.parent.path)

                        #Apply the move and store it
                        move = newnode.board.moveCar(c.letter, paths[i])
                        newnode.path.append(move)   
                        newnode.search = str(newnode.cost) + " " + str(newnode.cost) + " 0 " + move[4]

                        #If the item is already in the open list, skip it
                        #In other algos, need to check that cost is less
                        #For UCS, this will never be the case, since all costs are 1 in this case (breadth first search)
                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        #bisect.insort(self.opened, newnode)
                        self.opened.append(newnode)



# Building State Space Search - Greedy Best First Search (GBFS)

In [6]:
class HNode(object):
    def __init__(self, board, cost, parentCost):
        self.board = board
        self.cost = cost
        self.hCost = 0
        self.path = []
        self.parent = 0
        self.parentCost = parentCost
        self.search = ""
    def __lt__(self, other):
        return self.cost < other.cost
    def __eq__(self, other):
        return self.board == other.board
    
    def addHeuristic(self, heuristic):
        if heuristic == 1:
            self.cost = self.cost + self.board.h1()
        elif heuristic == 2:
            self.cost = self.cost + self.board.h2()
        elif heuristic == 3:
            self.cost = self.cost + self.board.h3(3)
        elif heuristic == 4:
            self.cost = self.cost + self.board.h4()
        return self.cost
    def setCost(self, cost):
        self.cost = cost
        
        
class GBFS(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0
        
    def next(self):
        self.count += 1
    
    def recursivePrinting(self, node):
        if node.cost != 0:
            self.recursivePrinting(node.parent)  
        print(node.board)
        
    def search(self, board, heuristic):
        self.opened = []
        self.closed = []
        
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        self.opened.append(a)
        a.addHeuristic(heuristic)
        a.search = str(a.cost) + " 0 " + str(a.cost) + " " + a.board.stringLine()
        
        start_time = time.time()
        statessearched = 0
        
        f = open("Results\\gbfs-h"+str(heuristic)+"-search-"+ str(self.count) +".txt", "w")
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\gbfs-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                
                with open("Results\\all_results.txt", "a") as f:
                    f.write(str(self.count) + "\t\t\tGBFS\t\th" + str(heuristic) + "\t\tNA\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close()
                
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
            
            #print(curr.search)
            f.write(curr.search + "\n")
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
                
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                #self.recursivePrinting(curr)
                print("Solution path: ")
                for move in curr.path:
                    print(move[0]+ " "+move[1] + " " + str(move[2])+"; ", end='')
                    
                ######
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\gbfs-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....
                f.write("\nSolution path: ")
                for move in curr.path:
                    f.write(move[0]+ " "+move[1] + " " + str(move[2])+"; ")

                f.write("\n\n");
                for move in curr.path:
                    f.write(move[0]+ "\t"+move[1] + "\t" + str(move[2]) + "\t" + str(move[3]) + "\t" + str(move[4])+"\n")

                f.write("\n" + str(curr.board) + "\n")
                f.close()
                
                with open("Results\\all_results.txt", "a") as f:
                    f.write(str(self.count) + "\t\t\tGBFS\t\th" + str(heuristic) + "\t\t" + str(curr.cost) + "\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close()
                
                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), 0, curr.cost)   
                        newnode.parent = curr
                        newnode.path = copy.deepcopy(newnode.parent.path)
                
                        #Apply the move and store it
                        move = newnode.board.moveCar(c.letter, paths[i])
                        newnode.path.append(move)
                        gCost = newnode.cost
                        newnode.addHeuristic(heuristic)
                        newnode.search = str(newnode.cost) + " " + str(gCost) + " " + str(newnode.cost - gCost) + " " + move[4] 
                    
                        #If the node is already in the open or closed list, skip it
                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        bisect.insort(self.opened, newnode)

# Building State Space Search - Algorithm A or A* (A/A*)

In [7]:
class AAStar(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0
        
    def next(self):
        self.count += 1
    
    def recursivePrinting(self, node):
        if node.cost != 0:
            self.recursivePrinting(node.parent)  
        print(node.board)
        
    def search(self, board, heuristic):
        self.opened = []
        self.closed = []
        
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        self.opened.append(a)
        a.addHeuristic(heuristic)
        a.search = str(a.cost) + " 0 " + str(a.cost) + " " + a.board.stringLine()
        
        start_time = time.time()
        statessearched = 0
        
        f = open("Results\\a-h"+str(heuristic)+"-search-"+ str(self.count) +".txt", "w")
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\a-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                
                with open("Results\\all_results.txt", "a") as f:
                    f.write(str(self.count) + "\t\t\tA/A*\t\th" + str(heuristic) + "\t\tNA\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close()
                
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
            
            #print(curr.search)
            f.write(curr.search + "\n")
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
            
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                print("Solution path: ")
                for move in curr.path:
                    print(move[0]+ " "+move[1] + " " + str(move[2])+"; ", end='')
                    
                ######
                
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\a-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....
                f.write("\nSolution path: ")
                for move in curr.path:
                    f.write(move[0]+ " "+move[1] + " " + str(move[2])+"; ")

                f.write("\n\n");
                for move in curr.path:
                    f.write(move[0]+ "\t"+move[1] + "\t" + str(move[2]) + "\t" + str(move[3]) + "\t" + str(move[4])+"\n")
                f.write("\n" + str(curr.board) + "\n")
                f.close()
                
                with open("Results\\all_results.txt", "a") as f:
                    f.write(str(self.count) + "\t\t\tA\A*\t\th" + str(heuristic) + "\t\t" + str(curr.cost) + "\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close()
                
                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), curr.cost+1, curr.cost)   
                        newnode.parent = curr
                        newnode.path = copy.deepcopy(newnode.parent.path)
                
                        #Apply the move and store it
                        move = newnode.board.moveCar(c.letter, paths[i])
                        newnode.path.append(move)
                        gCost = newnode.cost
                        newnode.addHeuristic(heuristic)
                        newnode.search = str(newnode.cost) + " " + str(gCost) + " " + str(newnode.cost - gCost) + " " + move[4]      
                    
                        #If the node is already in the open or closed list, skip if the cost in is higher, else replace
                        if newnode in self.opened:
                            for n in self.opened:
                                if n == newnode and n.cost > newnode.cost:
                                    self.opened.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        elif newnode in self.closed:
                            for n in self.closed:
                                if n == newnode and n.cost > newnode.cost:
                                    self.closed.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        else:
                            #Insert into open list (in sorted order)
                            bisect.insort(self.opened, newnode)

# Individual Runs

In [14]:
currentBoard = Board(boards[1].cars)
ucs = UCS()
ucs.search(currentBoard)

Initial board configuration

....ZZ
....O.
.AA.OB
....OB
.....C
...DDC

Car Fuel Available: Z:100, O:100, A:100, B:100, C:100, D:100, 
Runtime: 3.086444854736328 seconds
Number of moves:  5
Search path length: 665 states
------
Solution path: 
Z left 1; B up 2; D left 1; O down 2; A right 3; 

In [ ]:
currentBoard = Board(boards[47].cars)
gbfs = GBFS()
gbfs.search(currentBoard, 4)

In [ ]:
currentBoard = Board(boards[42].cars)
aastar = AAStar()
aastar.search(currentBoard, 4)

# Running the 50 simulations

In [8]:
f = open("Results\\all_results.txt", "w")
f.write("Puzzle number\tAlgorithm\tHeuristic\tLength of solution\tLength of search path\tExecution Time (seconds)\n")
f.close()

In [9]:
ucs = UCS()
gbfs = GBFS()
aastar = AAStar()

for b in boards:
    currentBoard = Board(b.cars) 
    ucs.search(currentBoard)
    gbfs.next()
    for i in range(1,5): 
        gbfs.search(currentBoard, i)
    aastar.next()
    for i in range(1,5): 
        aastar.search(currentBoard, i)
    f = open("Results\\all_results.txt", "a")
    f.write("\n")
    f.close()

Initial board configuration

Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

Car Fuel Available: Z:100, B:100, O:100, A:100, C:100, D:100, P:100, E:100, 
Runtime: 7.464373350143433 seconds
Number of moves:  9
Search path length: 1505 states
------
Solution path: 
Z down 3; A left 1; B down 1; O left 3; C up 2; E up 4; P right 1; B down 2; A right 4; Initial board configuration

Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

Car Fuel Available: Z:100, B:100, O:100, A:100, C:100, D:100, P:100, E:100, 
Runtime: 0.7526299953460693 seconds
Number of moves:  9
Search path length: 229 states
------
Solution path: 
Z down 3; A left 1; B down 1; O left 3; C up 2; E up 4; P right 1; B down 2; A right 4; Initial board configuration

Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E

Car Fuel Available: Z:100, B:100, O:100, A:100, C:100, D:100, P:100, E:100, 
Runtime: 0.7151002883911133 seconds
Number of moves:  9
Search path length: 229 states
------
Solution path: 
Z down 3; A left 1; B down 1; O left 3; C up 2;

B up 2; A right 2; O up 2; D left 2; E left 2; C down 1; A right 2; Initial board configuration

..ZOOO
..ZP.B
.AAP.B
...P..
.....C
.....C

Car Fuel Available: Z:100, O:100, P:100, B:100, A:100, C:100, 
Runtime: 1.2709074020385742 seconds
Number of moves:  6
Search path length: 440 states
------
Solution path: 
P down 2; A left 1; Z down 3; O left 1; B up 1; A right 4; Initial board configuration

..ZOOO
..ZP.B
.AAP.B
...P..
.....C
.....C

Car Fuel Available: Z:100, O:100, P:100, B:100, A:100, C:100, 
Runtime: 0.10199570655822754 seconds
Number of moves:  6
Search path length: 33 states
------
Solution path: 
P down 2; A left 1; Z down 3; O left 1; B up 1; A right 4; Initial board configuration

..ZOOO
..ZP.B
.AAP.B
...P..
.....C
.....C

Car Fuel Available: Z:100, O:100, P:100, B:100, A:100, C:100, 
Runtime: 0.07899785041809082 seconds
Number of moves:  6
Search path length: 33 states
------
Solution path: 
P down 2; A left 1; Z down 3; O left 1; B up 1; A right 4; Initial board config

Z up 1; B up 1; C up 1; P left 3; O down 3; A right 3; Initial board configuration

...O..
...O.Z
.AAO.Z
......
B.C...
B.CPPP

Car Fuel Available: O:100, Z:100, A:100, B:100, C:100, P:100, 
Runtime: 3.2221474647521973 seconds
Number of moves:  6
Search path length: 609 states
------
Solution path: 
Z up 1; B up 1; C up 1; P left 3; O down 3; A right 3; Initial board configuration

.....Z
..OOOZ
AA.B.C
...B.C
.DDPPP
......

Car Fuel Available: Z:100, O:100, A:100, B:100, C:100, D:100, P:100, 
Runtime: 0.13399982452392578 seconds
Number of moves:  6
Search path length: 83 states
------
Solution path: 
O left 2; B up 2; D left 1; P left 1; C down 1; A right 4; Initial board configuration

.....Z
..OOOZ
AA.B.C
...B.C
.DDPPP
......

Car Fuel Available: Z:100, O:100, A:100, B:100, C:100, D:100, P:100, 
Runtime: 0.026999473571777344 seconds
Number of moves:  6
Search path length: 17 states
------
Solution path: 
O left 2; B up 2; D left 1; P left 1; C down 1; A right 4; Initial board configur

A left 2; O up 2; F up 2; G left 4; D down 2; E down 2; O down 2; F down 1; A right 4; Initial board configuration

ZZBBCC
....DE
..AADE
..O...
..OFGG
..OF..

Car Fuel Available: Z:100, B:100, C:100, D:100, E:100, A:100, O:100, F:100, G:100, 
Runtime: 1.8154833316802979 seconds
Number of moves:  9
Search path length: 423 states
------
Solution path: 
A left 2; O up 2; F up 2; G left 4; D down 2; E down 2; O down 2; F down 1; A right 4; Initial board configuration

ZZBBCC
....DE
..AADE
..O...
..OFGG
..OF..

Car Fuel Available: Z:100, B:100, C:100, D:100, E:100, A:100, O:100, F:100, G:100, 
Runtime: 0.325164794921875 seconds
Number of moves:  12
Search path length: 121 states
------
Solution path: 
A left 2; F up 3; G left 1; E down 2; O up 2; G left 1; D down 2; G left 1; F down 2; G left 1; O down 2; A right 4; Initial board configuration

ZZBBCC
....DE
..AADE
..O...
..OFGG
..OF..

Car Fuel Available: Z:100, B:100, C:100, D:100, E:100, A:100, O:100, F:100, G:100, 
Runtime: 1.4104597568

E down 1; B down 1; C left 1; D up 2; Z left 2; C left 2; B up 2; A right 4; Initial board configuration

...ZZ.
...BCC
AA.B.D
...E.D
...EFF
......

Car Fuel Available: Z:100, B:100, C:100, A:100, D:100, E:100, F:100, 
Runtime: 0.9234490394592285 seconds
Number of moves:  7
Search path length: 277 states
------
Solution path: 
Z left 2; B up 1; E up 1; F left 3; D down 1; E down 1; A right 4; Initial board configuration

OZBBCC
OZ.DEE
OAADF.
GGH.FP
..HIIP
.JJKKP

Car Fuel Available: O:100, Z:100, B:100, C:100, D:100, E:100, A:100, F:100, G:100, H:100, P:100, I:100, J:100, K:100, 
Runtime: 313.0123805999756 seconds
Number of moves:  31
Search path length: 11130 states
------
Solution path: 
D down 1; E left 2; F up 1; P up 2; I right 1; J left 1; H down 1; K right 1; D down 2; A right 1; G right 3; Z down 3; H up 1; J right 1; O down 3; B left 2; C left 1; E left 2; A left 2; H up 3; P up 1; G right 1; D up 3; G left 2; I left 2; F down 2; K left 1; P down 3; C right 1; D up 1; A right 

...

C left 1; E up 2; G up 2; H up 2; O right 3; D down 2; A right 1; F right 1; Z down 3; A left 1; B down 1; C left 1; G up 2; C left 2; B up 1; A right 3; F right 2; D up 2; O left 1; H down 1; A right 1; Initial board configuration

Z.B.CC
Z.B...
AAD..E
FFD..E
OOO.GH
....GH

Car Fuel Available: Z:100, B:100, C:100, A:100, D:100, E:100, F:100, O:100, G:100, H:100, 
Runtime: 1.1038551330566406 seconds
Number of moves:  21
Search path length: 370 states
------
Solution path: 
C left 1; E up 2; G up 2; H up 2; O right 3; D down 2; A right 1; F right 1; Z down 3; A left 1; B down 1; C left 1; G up 2; C left 2; B up 1; A right 3; F right 2; D up 2; O left 1; H down 1; A right 1; Initial board configuration

Z.B.CC
Z.B...
AAD..E
FFD..E
OOO.GH
....GH

Car Fuel Available: Z:100, B:100, C:100, A:100, D:100, E:100, F:100, O:100, G:100, H:100, 
Runtime: 2.9984076023101807 seconds
Number of moves:  19
Search path length: 815 states
------
Solution path: 
C left 1; E up 2; G up 3; H up 2; O right 3;

B up 1; D left 3; C down 1; E left 1; O down 3; A right 4; Initial board configuration

..ZZOP
.QQQOP
.BAAOP
.B.C..
...CDD
...RRR

Car Fuel Available: Z:100, O:100, P:100, Q:100, B:100, A:100, C:100, D:100, R:100, 
Runtime: 102.8211178779602 seconds
Number of moves:  10
Search path length: 5684 states
------
Solution path: 
Z left 1; Q left 1; B down 1; A left 1; C up 3; D left 2; R left 2; O down 3; P down 3; A right 3; Initial board configuration

..ZZOP
.QQQOP
.BAAOP
.B.C..
...CDD
...RRR

Car Fuel Available: Z:100, O:100, P:100, Q:100, B:100, A:100, C:100, D:100, R:100, 
Runtime: 7.129093885421753 seconds
Number of moves:  12
Search path length: 878 states
------
Solution path: 
Z left 1; Q left 1; B down 1; A left 1; C up 3; D left 1; R left 1; P down 3; D left 1; R left 1; O down 3; A right 3; Initial board configuration

..ZZOP
.QQQOP
.BAAOP
.B.C..
...CDD
...RRR

Car Fuel Available: Z:100, O:100, P:100, Q:100, B:100, A:100, C:100, D:100, R:100, 
Runtime: 6.293639898300171 seconds

Z left 2; C up 1; D right 2; O down 3; P down 3; A right 4; Initial board configuration

..OP..
..OP..
AAOP..
..ZZ..
.B..C.
.BDDC.

Car Fuel Available: O:100, P:100, A:100, Z:100, B:100, C:100, D:100, 
Runtime: 0.9497363567352295 seconds
Number of moves:  6
Search path length: 289 states
------
Solution path: 
Z left 2; C up 1; D right 2; O down 3; P down 3; A right 4; Initial board configuration

..OP..
..OP..
AAOP..
..ZZ..
.B..C.
.BDDC.

Car Fuel Available: O:100, P:100, A:100, Z:100, B:100, C:100, D:100, 
Runtime: 0.1184244155883789 seconds
Number of moves:  7
Search path length: 48 states
------
Solution path: 
Z left 1; C up 1; D right 2; P down 3; Z left 1; O down 3; A right 4; Initial board configuration

..OP..
..OP..
AAOP..
..ZZ..
.B..C.
.BDDC.

Car Fuel Available: O:100, P:100, A:100, Z:100, B:100, C:100, D:100, 
Runtime: 0.9866852760314941 seconds
Number of moves:  6
Search path length: 289 states
------
Solution path: 
Z left 2; C up 1; D right 2; O down 3; P down 3; A righ

C left 2; D left 2; P left 2; B down 2; A right 4; Initial board configuration

.ZZOOO
....B.
AA..B.
..CCDD
...PPP
......

Car Fuel Available: Z:2, O:100, B:100, A:100, C:100, D:100, P:100, 
Runtime: 0.23700785636901855 seconds
Number of moves:  5
Search path length: 95 states
------
Solution path: 
C left 2; D left 2; P left 2; B down 2; A right 4; Initial board configuration

.ZZOOO
....B.
AA..B.
..CCDD
...PPP
......

Car Fuel Available: Z:2, O:100, B:100, A:100, C:100, D:100, P:100, 
Runtime: 0.10991692543029785 seconds
Number of moves:  5
Search path length: 40 states
------
Solution path: 
C left 2; D left 2; P left 2; B down 2; A right 4; Initial board configuration

.ZZOOO
....B.
AA..B.
..CCDD
...PPP
......

Car Fuel Available: Z:2, O:100, B:100, A:100, C:100, D:100, P:100, 
Runtime: 0.47132372856140137 seconds
Number of moves:  5
Search path length: 168 states
------
Solution path: 
C left 2; D left 2; P left 2; B down 2; A right 4; Initial board configuration

.ZZOOO
....B.
AA

C up 1; Z up 1; B up 1; A left 2; D down 1; E up 3; P up 3; F left 4; O down 3; P down 3; A right 4; Initial board configuration

OOO.PQ
....PQ
..AAPQ
......
..ZRRR
..ZBB.

Car Fuel Available: O:10, P:10, Q:10, A:10, Z:4, R:100, B:1, 
Runtime: 3.4464964866638184 seconds
Number of moves:  9
Search path length: 974 states
------
Solution path: 
A left 2; Z up 2; R left 2; Q down 3; B left 1; P down 3; O right 3; Z up 2; A right 4; Initial board configuration

OOO.PQ
....PQ
..AAPQ
......
..ZRRR
..ZBB.

Car Fuel Available: O:10, P:10, Q:10, A:10, Z:4, R:100, B:1, 
Runtime: 0.42987632751464844 seconds
Number of moves:  11
Search path length: 187 states
------
Solution path: 
P down 1; Q down 1; O right 3; A left 2; Z up 4; R left 1; Q down 2; R left 1; B left 1; P down 2; A right 4; Initial board configuration

OOO.PQ
....PQ
..AAPQ
......
..ZRRR
..ZBB.

Car Fuel Available: O:10, P:10, Q:10, A:10, Z:4, R:100, B:1, 
Runtime: 0.42633795738220215 seconds
Number of moves:  11
Search path length: